## Generate a knowledge graph and question-answer pairs from a PDF file

In [ ]:
import os
from dotenv import load_dotenv
from haystack import Pipeline
from haystack.components.converters import PyPDFToDocument
from haystack.components.preprocessors import (
    DocumentCleaner,
    DocumentSplitter)
from pathlib import Path
from scripts.knowledge_graph_component import KnowledgeGraphGenerator
from scripts.synthetic_test_components import SyntheticTestGenerator,\
                                                TestDatasetSaver,\
                                                    DocumentToLangChainConverter

# Load environment variables
load_dotenv("./.env")

# Example: Create a complete pipeline for synthetic test generation
data_path = "data_for_indexing"

if os.path.exists(data_path):
    print("Creating synthetic test generation pipeline...")
    
    # Get PDF files from the directory
    pdf_files = list(Path(data_path).glob("*.pdf"))
    
    if pdf_files:
        print(f"Found {len(pdf_files)} PDF files to process")
        
        # Create pipeline components
        pdf_converter = PyPDFToDocument()
        doc_cleaner = DocumentCleaner(remove_empty_lines=True,
                                      remove_extra_whitespaces=True)
        doc_splitter = DocumentSplitter(split_by="sentence",
                                       split_length=50,
                                       split_overlap=5)
        doc_converter = DocumentToLangChainConverter()
        kg_generator = KnowledgeGraphGenerator(apply_transforms=True)
        
        
        test_generator = SyntheticTestGenerator(
            testset_size=10,  
            llm_model="gpt-4o-mini",
            query_distribution=[
                ("single_hop", 0.25), 
                ("multi_hop_specific", 0.25),
                ("multi_hop_abstract", 0.5)
            ],
            # Optional: Add max_testset_size=5 if you want to limit due to API constraints
            # max_testset_size=5  # Uncomment this line if you experience API timeouts
        )
        test_saver = TestDatasetSaver("data_for_eval/synthetic_tests_10_from_pdf.csv")
        
        # Create pipeline
        pipeline = Pipeline()
        pipeline.add_component("pdf_converter", pdf_converter)
        pipeline.add_component("doc_cleaner", doc_cleaner)
        pipeline.add_component("doc_splitter", doc_splitter)
        pipeline.add_component("doc_converter", doc_converter)
        pipeline.add_component("kg_generator", kg_generator)
        pipeline.add_component("test_generator", test_generator)
        pipeline.add_component("test_saver", test_saver)
        
        # Connect components in sequence
        pipeline.connect("pdf_converter.documents", "doc_cleaner.documents")
        pipeline.connect("doc_cleaner.documents", "doc_splitter.documents")
        pipeline.connect("doc_splitter.documents", "doc_converter.documents")
        pipeline.connect("doc_converter.langchain_documents", "kg_generator.documents")
        pipeline.connect("kg_generator.knowledge_graph", "test_generator.knowledge_graph")
        pipeline.connect("doc_converter.langchain_documents", "test_generator.documents")
        pipeline.connect("test_generator.testset", "test_saver.testset")
        
        # Prepare input data - convert PDF files to ByteStream objects
        pdf_sources = [Path("./data_for_indexing/howpeopleuseai.pdf")]
         
        result = pipeline.run({
            "pdf_converter": {"sources": pdf_sources}
        })
        
        print("\n📊 Pipeline Results:")
        print(f"  📄 Documents Processed: {result['doc_converter']['document_count']}")
        print(f"  🧠 Knowledge Graph Nodes: {result['kg_generator']['node_count']}")
        print(f"  🧪 Test Cases Generated: {result['test_generator']['testset_size']}")
        print(f"  🔧 Generation Method: {result['test_generator']['generation_method']}")
        
    else:
        print("❌ No PDF files found in data_for_indexing directory")
else:
    print("❌ Data path 'data_for_indexing' not found")

Creating synthetic test generation pipeline...
Found 1 PDF files to process


Generating Samples: 100%|██████████| 11/11 [00:08<00:00,  1.37it/s]



📊 Pipeline Results:
  📄 Documents Processed: 17
  🧠 Knowledge Graph Nodes: 17
  🧪 Test Cases Generated: 11
  🔧 Generation Method: knowledge_graph


In [17]:
import pandas as pd

# Load and display the generated synthetic tests
test_file_path = "data_for_eval/synthetic_tests_10_from_pdf.csv"

if os.path.exists(test_file_path):
    synthetic_tests_df = pd.read_csv(test_file_path)
    print("\n🧪 Synthetic Tests Sample:")
    print("First 5 rows:")
    display(synthetic_tests_df.head())
    print("Last 5 rows:")
    display(synthetic_tests_df.tail())
else:
    print("❌ Synthetic test file not found")


🧪 Synthetic Tests Sample:
First 5 rows:


,user_input,reference_contexts,reference,synthesizer_name
0,Who is Zoe Hitzig and what is her role in the ...,['NBER WORKING PAPER SERIES\nHOW PEOPLE USE CH...,Zoe Hitzig is one of the co-authors of the NBE...,single_hop_specific_query_synthesizer
1,How many users was ChatGPT having by July 2025?,['ABSTRACT Despite the rapid adoption of LLM c...,"By July 2025, ChatGPT had 700 million users, r...",single_hop_specific_query_synthesizer
2,What insights does Roth provide regarding the ...,"['LLM, allowing us to classify messages withou...",Roth (2025) reports that 28% of US adults used...,single_hop_specific_query_synthesizer
3,What trends in user engagement and message vol...,['<1-hop>\n\nThe yellow line represents the fi...,"For ChatGPT users who signed up in 2023, parti...",multi_hop_specific_query_synthesizer
4,What trends in user interaction quality and ge...,['<1-hop>\n\n5.5 Quality of Interactions\nWe a...,"In the lead-up to June 2025, trends in user in...",multi_hop_specific_query_synthesizer


Last 5 rows:


,user_input,reference_contexts,reference,synthesizer_name
6,What trends can be observed in ChatGPT user co...,['<1-hop>\n\nThe yellow line represents the fi...,The trends observed in ChatGPT user cohorts in...,multi_hop_abstract_query_synthesizer
7,How does user satisfaction relate to data priv...,['<1-hop>\n\nWe retain this classifier because...,User satisfaction in ChatGPT interactions is a...,multi_hop_abstract_query_synthesizer
8,What is the significance of IWA ID in relation...,['<1-hop>\n\nTask details Your response should...,The significance of IWA ID in relation to Cohe...,multi_hop_abstract_query_synthesizer
9,What are the key patterns of ChatGPT usage amo...,['<1-hop>\n\nNBER WORKING PAPER SERIES\nHOW PE...,The key patterns of ChatGPT usage reveal that ...,multi_hop_abstract_query_synthesizer
10,What insights can be drawn about user satisfac...,"['<1-hop>\n\nOuyang, Long, Jeff Wu, Xu Jiang, ...",Insights about user satisfaction from the feed...,multi_hop_abstract_query_synthesizer


## Generate a knowledge graph and question-answer pairs from a scraped website

In [15]:
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument

fetcher = LinkContentFetcher()
converter = HTMLToDocument()
doc_cleaner = DocumentCleaner(remove_empty_lines=True,
                                      remove_extra_whitespaces=True)
doc_splitter = DocumentSplitter(split_by="sentence",
                                split_length=50,
                                split_overlap=5)
doc_converter = DocumentToLangChainConverter()
kg_generator = KnowledgeGraphGenerator(apply_transforms=True)
test_generator = SyntheticTestGenerator(
            testset_size=10,  
            llm_model="gpt-4o-mini",
            query_distribution=[
                ("single_hop", 0.25), 
                ("multi_hop_specific", 0.25),
                ("multi_hop_abstract", 0.5)
            ]
        )
test_saver = TestDatasetSaver("data_for_eval/synthetic_tests_10_from_html_page.csv")

# Create pipeline
pipeline = Pipeline()
pipeline.add_component("fetcher", fetcher)
pipeline.add_component("converter", converter)
pipeline.add_component("doc_cleaner", doc_cleaner)
pipeline.add_component("doc_splitter", doc_splitter)
pipeline.add_component("doc_converter", doc_converter)
pipeline.add_component("kg_generator", kg_generator)
pipeline.add_component("test_generator", test_generator)
pipeline.add_component("test_saver", test_saver)

# Connect components in sequence
pipeline.connect("fetcher.streams", "converter.sources")
pipeline.connect("converter.documents", "doc_cleaner.documents")
pipeline.connect("doc_cleaner.documents", "doc_splitter.documents")
pipeline.connect("doc_splitter.documents", "doc_converter.documents")
pipeline.connect("doc_converter.langchain_documents", "kg_generator.documents")
pipeline.connect("kg_generator.knowledge_graph", "test_generator.knowledge_graph")
pipeline.connect("doc_converter.langchain_documents", "test_generator.documents")
pipeline.connect("test_generator.testset", "test_saver.testset")

web_url = "https://haystack.deepset.ai/blog/haystack-2-release"

result = pipeline.run({
    "fetcher": {"urls": [web_url]}
})

print("\n📊 Pipeline Results:")
print(f"  📄 Documents Processed: {result['doc_converter']['document_count']}")
print(f"  🧠 Knowledge Graph Nodes: {result['kg_generator']['node_count']}")
print(f"  🧪 Test Cases Generated: {result['test_generator']['testset_size']}")
print(f"  🔧 Generation Method: {result['test_generator']['generation_method']}")

Generating Samples: 100%|██████████| 11/11 [00:07<00:00,  1.57it/s]



📊 Pipeline Results:
  📄 Documents Processed: 2
  🧠 Knowledge Graph Nodes: 2
  🧪 Test Cases Generated: 11
  🔧 Generation Method: knowledge_graph


In [18]:
# Load and display the generated synthetic tests
test_file_path = "data_for_eval/synthetic_tests_10_from_html_page.csv"

if os.path.exists(test_file_path):
    synthetic_tests_df = pd.read_csv(test_file_path)
    print("\n🧪 Synthetic Tests Sample:")
    print("First 5 rows:")
    display(synthetic_tests_df.head())
    print("Last 5 rows:")
    display(synthetic_tests_df.tail())
else:
    print("❌ Synthetic test file not found")


🧪 Synthetic Tests Sample:
First 5 rows:


,user_input,reference_contexts,reference,synthesizer_name
0,Wen was Haystack first released?,['Haystack 2.0: The Composable Open-Source LLM...,Haystack was first officially released in 2020.,single_hop_specific_query_synthesizer
1,How does Haystack 2.0 improve upon the limitat...,['Composable and customizable Pipelines\nModer...,Haystack 2.0 improves upon the limitations of ...,single_hop_specific_query_synthesizer
2,What are the key features and benefits of Asse...,['A common interface for storing data - A clea...,Assembly AI contributes to the Haystack ecosys...,single_hop_specific_query_synthesizer
3,What are the limitations of Haystack 1.0 regar...,['<1-hop>\n\nHaystack 2.0: The Composable Open...,Haystack 1.0 had a significant limitation in t...,multi_hop_specific_query_synthesizer
4,What limitations of Haystack 1.0 were addresse...,['<1-hop>\n\nA common interface for storing da...,One important limitation in Haystack 1.0 was t...,multi_hop_specific_query_synthesizer


Last 5 rows:


,user_input,reference_contexts,reference,synthesizer_name
6,How does Haystack 2.0 enhance the integration ...,['<1-hop>\n\nHaystack 2.0: The Composable Open...,Haystack 2.0 enhances the integration of data ...,multi_hop_abstract_query_synthesizer
7,What are the main features of Haystack 2.0 tha...,['<1-hop>\n\nHaystack 2.0: The Composable Open...,Haystack 2.0 introduces several main features ...,multi_hop_abstract_query_synthesizer
8,How does Haystack 2.0 facilitate integration w...,['<1-hop>\n\nHaystack 2.0: The Composable Open...,Haystack 2.0 facilitates integration with mode...,multi_hop_abstract_query_synthesizer
9,What are the key features of Haystack 2.0 that...,['<1-hop>\n\nHaystack 2.0: The Composable Open...,Haystack 2.0 introduces several key features t...,multi_hop_abstract_query_synthesizer
10,How does Haystack 2.0 integrate with data stor...,['<1-hop>\n\nHaystack 2.0: The Composable Open...,Haystack 2.0 integrates with data storage serv...,multi_hop_abstract_query_synthesizer
